In [1]:
import os, json, pandas as pd, traceback

In [14]:
# !pip install -U langchain-openai

In [2]:
#from langchain.chat_models import ChatOpenAI
# LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. 
# An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
from langchain_openai import ChatOpenAI


In [25]:
import os
from dotenv import load_dotenv

load_dotenv() # taking env variables from .env file. 

mykey=os.environ["OPENAI_API_KEY"]


In [5]:
mykey=os.environ["OPENAI_API_KEY"]
# temperature   0 - more determinisic.   2 - more randomness 
llm=ChatOpenAI(openai_api_key=mykey, model_name="gpt-3.5-turbo", temperature=0.5)

In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2 


In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [8]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [10]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [11]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [13]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [14]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [15]:
file_path=r"G:\My Drive\backup jerry-200k(master copy)\AI\project_MCQ\data_geometry.txt"

with open(file_path, 'r') as file: 
    TEXT=file.read() 
    

In [16]:
# serialize the python dictionary into a JSON-formatted string. 
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [17]:
NUMBER=5
SUBJECT="Geometry" 
TONE="Difficult"

In [18]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

c:\Users\jerry\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:High school geometry typically covers a range of topics designed to introduce students to the fundamental concepts and principles of geometry. The scope includes both theoretical understanding and practical applications. Here are the key areas usually covered in high school geometry:

1. Basic Geometric Concepts
Points, lines, line segments, rays, and planes
Angles (acute, right, obtuse, and straight)
Parallel and perpendicular lines
2. Triangles
Types of triangles (equilateral, isosceles, scalene)
Triangle properties and theorems (Pythagorean theorem, Triangle inequality theorem)
Congruence and similarity (criteria like SSS, SAS, ASA, and AA)
3. Quadrilaterals and Polygons
Properties of quadrilaterals (parallelograms, rectangles, squares, rhombuses, trapezoids)
Classification and properties of polygons
Interior and exterior angles of polygons
4. Circles
Properties of circles (ra

In [19]:
response


{'text': 'High school geometry typically covers a range of topics designed to introduce students to the fundamental concepts and principles of geometry. The scope includes both theoretical understanding and practical applications. Here are the key areas usually covered in high school geometry:\n\n1. Basic Geometric Concepts\nPoints, lines, line segments, rays, and planes\nAngles (acute, right, obtuse, and straight)\nParallel and perpendicular lines\n2. Triangles\nTypes of triangles (equilateral, isosceles, scalene)\nTriangle properties and theorems (Pythagorean theorem, Triangle inequality theorem)\nCongruence and similarity (criteria like SSS, SAS, ASA, and AA)\n3. Quadrilaterals and Polygons\nProperties of quadrilaterals (parallelograms, rectangles, squares, rhombuses, trapezoids)\nClassification and properties of polygons\nInterior and exterior angles of polygons\n4. Circles\nProperties of circles (radius, diameter, circumference, area)\nArcs, chords, tangents, and secants\nAngle re

In [21]:
print(response["review"])


{
"1": {
"mcq": "What is the sum of the interior angles of a triangle?",
"options": {"a": "180 degrees", "b": "90 degrees", "c": "270 degrees", "d": "360 degrees"},
"correct": "a"
},
"2": {
"mcq": "Which of the following is NOT a type of quadrilateral?",
"options": {"a": "Parallelogram", "b": "Equilateral triangle", "c": "Rectangle", "d": "Rhombus"},
"correct": "b"
},
"3": {
"mcq": "What is the equation of a line in slope-intercept form?",
"options": {"a": "y = mx + b", "b": "y = ax^2 + bx + c", "c": "y = a/x + b", "d": "y = x^2 + y^2"},
"correct": "a"
},
"4": {
"mcq": "Which trigonometric ratio is equal to opposite/hypotenuse in a right triangle?",
"options": {"a": "Sine", "b": "Cosine", "c": "Tangent", "d": "Secant"},
"correct": "a"
},
"5": {
"mcq": "What is the formula for calculating the volume of a cone?",
"options": {"a": "V = 1/3 * π * r^2 * h", "b": "V = π * r^2", "c": "V = 2/3 * π * r^3", "d": "V = 4/3 * π * r^3"},
"correct": "a"
}
}


In [22]:
print ( SUBJECT)

Geometry


In [85]:
from datetime import datetime 
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'05_29_2024_16_40_33'

In [86]:
import os 
os.getcwd()

'g:\\My Drive\\backup jerry-200k(master copy)\\AI\\project_MCQ\\experiment'